In [1]:
import keras_applications
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import boto3, os, tempfile
from botocore.exceptions import ClientError

Using TensorFlow backend.


In [2]:
# enter the name of S3 bucket where your model is stored
MODEL_BUCKET_NAME = '' 

# enter model file name
MODEL_KEY_NAME = 'inception_v3_weights_tf_dim_ordering_tf_kernels.h5'

# enter the name of S3 bucket where images will be uploaded
IMG_UPLOAD_BUCKET_NAME = ''

# uploaded image file name example
IMG_KEY_NAME = 'elephant.jpg'

In [3]:
temp_dir = 'tmp'

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

s3 = boto3.resource('s3')

In [ ]:
%%time 
# this cell will download the model from your S3 bucket if it's uploaded there
model = None
model_path = os.path.join(temp_dir, MODEL_KEY_NAME)
try:
    print('Downloading model from S3...')
    s3.Bucket(MODEL_BUCKET_NAME).download_file(MODEL_KEY_NAME, model_path)
    print('Model downloaded to', model_path)

# if the model can't be downloaded from S3 bucket, it will be loaded from the home/.keras/models directory
# if the model isn't awailable from home/.keras/models directory, it will be downloaded 
# from official location and loaded to memory
except ClientError as error: 
    print(error)
    print('Loading model from official destination...')
    model = InceptionV3()
    print('Model loaded.')

In [ ]:
%%time
# if the model is downloaded from S3 bucket, it will be loaded to memory
if model is None:
    print('Loading model...')
    model = InceptionV3(weights=model_path)
    print('Model loaded.')

In [ ]:
print('Downloading image...')
tmp_file = tempfile.NamedTemporaryFile(dir=temp_dir, delete=False)
img_object = s3.Bucket(IMG_UPLOAD_BUCKET_NAME).Object(IMG_KEY_NAME)
img_object.download_fileobj(tmp_file)
tmp_file.close()
print('Image downloaded to', tmp_file.name)

In [7]:
image_path = tmp_file.name

In [8]:
img = image.load_img(image_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) 
x = preprocess_input(x)

In [ ]:
print('Predicting...')
preds = model.predict(x)
print(preds.shape)

In [ ]:
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
keras_applications.imagenet_utils.CLASS_INDEX